# Longest Common Subsequence

Заданы две строки s1 и s2. Требуется найти длину самой длинной общей подпоследовательности. Если общей подпоследовательности нет - выдать 0.

Подпоследовательность - это строка, сгенерированная из исходной строки путем удаления 0 или более символов и без изменения относительного порядка остальных символов.

Например, подпоследовательностями “ABC” являются “”, “A”, “B”, “C”, “AB”, “AC”, “BC” и “ABC”.

В общем случае, строка длины $n$ имеет $2^n$ подпоследовательностей.

"Наивный" алгоритм. Время - $O(2^{\min(m,n)})$, память - $O(\min(m, n))$

In [1]:
# A Naive recursive implementation of LCS problem

# Returns length of LCS for s1[1:i], s2[1:j]
function lcs(i, j, s1, s2)
    if (i == 0 || j == 0)
        0
    elseif (s1[i] == s2[j])
        1 + lcs(i - 1, j - 1, s1, s2)
    else
        max(lcs(i, j - 1, s1, s2), lcs(i - 1, j, s1, s2))
    end
end

lcs(s1, s2) = lcs(length(s1), length(s2), s1, s2)

lcs (generic function with 2 methods)

In [2]:
using Test

In [3]:

@test lcs("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs("AGGTAB", "GXTXAYB") == 4
@test lcs("ABC", "CBA") == 1

Test Passed

## Специализация по `i` и `j`

- Peter Thiemann. 1999. **Combinators for program generation.** J. Funct. Program. 9, 5 (September 1999), 483–525. https://doi.org/10.1017/S0956796899003469

- Kedar Swadi, Walid Taha, Oleg Kiselyov. 2005. **Staging dynamic programming algorithms**. Unpublished manuscript (April 2005), available from: <http://www.cs.rice.edu/~taha/publications.html>.

- Yukiyoshi Kameyama, Oleg Kiselyov, and Chung-chieh Shan. 2009. **Shifting the stage: staging with delimited control.** In Proceedings of the 2009 ACM SIGPLAN workshop on Partial evaluation and program manipulation (PEPM '09). Association for Computing Machinery, New York, NY, USA, 111–120. <https://doi.org/10.1145/1480945.1480962>

- Oleg Kiselyov. 2010. **Delimited control in OCaml, abstractly and concretely: system description.** In Proceedings of the 10th international conference on Functional and Logic Programming (FLOPS'10). Springer-Verlag, Berlin, Heidelberg, 304–320. https://doi.org/10.1007/978-3-642-12251-4_22

In [4]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

"Наивный" генератор. Получается одно громадное выражение, в котором есть совпадающие подвыражения.

In [5]:
function lcs_gen_impl1(i, j)
    if (i == 0 || j == 0)
        0
    else
        quote
            if (s1[$i] == s2[$j])
                1 + $(lcs_gen_impl1(i - 1, j - 1))
            else
                max($(lcs_gen_impl1(i, j - 1)), $(lcs_gen_impl1(i - 1, j)))
            end
        end
    end
end

lcs_gen_impl1 (generic function with 1 method)

In [6]:
lcs_gen_impl1(2, 3) |> cleanup

:(if s1[2] == s2[3]
      1 + if s1[1] == s2[2]
              1 + 0
          else
              max(if s1[1] == s2[1]
                      1 + 0
                  else
                      max(0, 0)
                  end, 0)
          end
  else
      max(if s1[2] == s2[2]
              1 + if s1[1] == s2[1]
                      1 + 0
                  else
                      max(0, 0)
                  end
          else
              max(if s1[2] == s2[1]
                      1 + 0
                  else
                      max(0, if s1[1] == s2[1]
                              1 + 0
                          else
                              max(0, 0)
                          end)
                  end, if s1[1] == s2[2]
                      1 + 0
                  else
                      max(if s1[1] == s2[1]
                              1 + 0
                          else
                              max(0, 0)
                          end, 0)
                  

Не очень хорошо, когда выражения получаются слишком большими. (Это может "взорвать" компилятор, и читать трудно.)

Посему, введём переменные для промежуточных результатов.

Ну, а ещё - введение переменных - это подготовка мемоизации.

In [7]:
function lcs_gen_impl2(i, j)
    if (i == 0 || j == 0)
        0
    else
        r0 = Symbol("r_", i - 1, "_", j - 1)
        r1 = Symbol("r_", i, "_", j - 1)
        r2 = Symbol("r_", i - 1, "_", j)
        quote
            if (s1[$i] == s2[$j])
                let
                    local $r0 = $(lcs_gen_impl2(i - 1, j - 1))
                    1 + $r0
                end
            else
                let
                    local $r1 = $(lcs_gen_impl2(i, j - 1))
                    local $r2 = $(lcs_gen_impl2(i - 1, j))
                    max($r1, $r2)
                end
            end
        end
    end
end

lcs_gen_impl2 (generic function with 1 method)

In [8]:
lcs_gen_impl2(2, 3) |> cleanup

:(if s1[2] == s2[3]
      let
          local r_1_2 = if s1[1] == s2[2]
                      let
                          local r_0_1 = 0
                          1 + r_0_1
                      end
                  else
                      let
                          local r_1_1 = if s1[1] == s2[1]
                                      let
                                          local r_0_0 = 0
                                          1 + r_0_0
                                      end
                                  else
                                      let
                                          local r_1_0 = 0
                                          local r_0_1 = 0
                                          max(r_1_0, r_0_1)
                                      end
                                  end
                          local r_0_2 = 0
                          max(r_1_1, r_0_2)
                      end
                  end
          1 + r_1_2
      

Недостаток сгенерированной программы в том, что одни и те же выражения повторяются в программе несколько раз.

In [9]:
@generated function lcs_gen2(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl2(i, j)
end

lcs_gen2 (generic function with 1 method)

In [10]:
function lcs_gen2(s1, s2)
    lcs_gen2(Val(length(s1)), Val(length(s2)), s1, s2)
end

lcs_gen2 (generic function with 2 methods)

In [11]:
@test lcs_gen2("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
# @test lcs_gen2("AGGTAB", "GXTXAYB") == 4
@test lcs_gen2("ABC", "CBA") == 1

Test Passed

Можно вынести присваивания из-под `if (s1[i] == s2[j])`. Тогда будут выполняться лишние вычисления, но асимптотика от этого не изменится.

In [12]:
function lcs_gen_impl3(i, j)
    if (i == 0 || j == 0)
        0
    else
        r0 = Symbol("r_", i - 1, "_", j - 1)
        r1 = Symbol("r_", i, "_", j - 1)
        r2 = Symbol("r_", i - 1, "_", j)
        quote
            $r0 = $(lcs_gen_impl3(i - 1, j - 1))
            $r1 = $(lcs_gen_impl3(i, j - 1))
            $r2 = $(lcs_gen_impl3(i - 1, j))
            if (s1[$i] == s2[$j])
                1 + $r0
            else
                max($r1, $r2)
            end
        end
    end
end

lcs_gen_impl3 (generic function with 1 method)

In [13]:
lcs_gen_impl3(2, 3) |> cleanup

quote
    r_1_2 = begin
            r_0_1 = 0
            r_1_1 = begin
                    r_0_0 = 0
                    r_1_0 = 0
                    r_0_1 = 0
                    if s1[1] == s2[1]
                        1 + r_0_0
                    else
                        max(r_1_0, r_0_1)
                    end
                end
            r_0_2 = 0
            if s1[1] == s2[2]
                1 + r_0_1
            else
                max(r_1_1, r_0_2)
            end
        end
    r_2_2 = begin
            r_1_1 = begin
                    r_0_0 = 0
                    r_1_0 = 0
                    r_0_1 = 0
                    if s1[1] == s2[1]
                        1 + r_0_0
                    else
                        max(r_1_0, r_0_1)
                    end
                end
            r_2_1 = begin
                    r_1_0 = 0
                    r_2_0 = 0
                    r_1_1 = begin
                            r_0_0 = 0
                       

Теперь недостаток в том, что одно и то же выражение вычисляется и присваивается несколько раз.

Кроме того, генерируются переменные, которым присваиваются константы.

Можно решить эти две проблемы, заведя словарь, в котором будет регистрироваться, какие подвыражения уже вычислялись, а какие - нет.

В результате получим асимптотику: время - $O(m * n)$, память - $O(m * n)$.

In [14]:
function ass!(es, d, s::Symbol, c::Int64)
    d[s] = c
end

function ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function ass!(es, d, s::Symbol, e)
    haskey(d, s) && return
    d[s] = s
    push!(es, :($s = $e))
end

ass! (generic function with 3 methods)

In [15]:
function lcs_gen_impl4!(es, d, i, j)
    r = Symbol("r_", i, "_", j)

    if haskey(d, r)
        return d[r]
    end

    if (i == 0 || j == 0)
        ass!(es, d, r, 0)
    else
        r0 = lcs_gen_impl4!(es, d, i - 1, j - 1)
        r1 = lcs_gen_impl4!(es, d, i, j - 1)
        r2 = lcs_gen_impl4!(es, d, i - 1, j)
        ass!(es, d, r, :(
            s1[$i] == s2[$j] ? 1 + $r0 : max($r1, $r2)
        ))
    end

    return d[r]
end

lcs_gen_impl4! (generic function with 1 method)

In [16]:
function lcs_gen_impl4(i, j)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = lcs_gen_impl4!(es, d, i, j)

    quote
        $(es...)
        return $r
    end
end

lcs_gen_impl4 (generic function with 1 method)

In [17]:
lcs_gen_impl4(2, 3) |> cleanup

quote
    r_1_1 = if s1[1] == s2[1]
            1 + 0
        else
            max(0, 0)
        end
    r_1_2 = if s1[1] == s2[2]
            1 + 0
        else
            max(r_1_1, 0)
        end
    r_2_1 = if s1[2] == s2[1]
            1 + 0
        else
            max(0, r_1_1)
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1 + 0
        else
            max(r_1_2, 0)
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    return r_2_3
end

In [18]:
@generated function lcs_gen4(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl4(i, j)
end

lcs_gen4 (generic function with 1 method)

In [19]:
function lcs_gen4(s1, s2)
    lcs_gen4(Val(length(s1)), Val(length(s2)), s1, s2)
end

lcs_gen4 (generic function with 2 methods)

In [20]:
@test lcs_gen4("ABC", "ACD") == 2
# The longest common subsequence is “GTAB”.
@test lcs_gen4("AGGTAB", "GXTXAYB") == 4
@test lcs_gen4("ABC", "CBA") == 1

Test Passed

## Упрощение выражений через переписывание (Metatheory.jl)

In [21]:
using Metatheory, Metatheory.Rewriters

In [22]:
opt_rules = @theory c c1 c2 x y begin

    x::Int64 + y::Int64 => x + y

    max(0, y) --> y
    max(x, 0) --> x

end;

In [23]:
strategy = (#= Fixpoint ∘ =# Postwalk ∘ Chain)
opt_expr(e) = strategy(opt_rules)(e)

opt_expr (generic function with 1 method)

In [24]:
@test opt_expr(:(2 + 3)) == 5
@test opt_expr(:(max(10, 0))) == 10
@test opt_expr(:(max(0, 20))) == 20
@test opt_expr(:(true ? 1 + 2 : 3 + 4)) == :(true ? 3 : 7)

Test Passed

In [25]:
function ass_o!(es, d, s::Symbol, e)
    ass!(es, d, s, opt_expr(e))
end

ass_o! (generic function with 1 method)

In [26]:
function lcs_gen_impl5!(es, d, i, j)
    r = Symbol("r_", i, "_", j)

    if haskey(d, r)
        return d[r]
    end

    if (i == 0 || j == 0)
        ass_o!(es, d, r, 0)
    else
        r0 = lcs_gen_impl5!(es, d, i - 1, j - 1)
        r1 = lcs_gen_impl5!(es, d, i, j - 1)
        r2 = lcs_gen_impl5!(es, d, i - 1, j)
        ass_o!(es, d, r, :(
            s1[$i] == s2[$j] ? 1 + $r0 : max($r1, $r2)
        ))
    end

    return d[r]
end

lcs_gen_impl5! (generic function with 1 method)

In [27]:
function lcs_gen_impl5(i, j)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = lcs_gen_impl5!(es, d, i, j)

    quote
        $(es...)
        return $r
    end
end

lcs_gen_impl5 (generic function with 1 method)

In [28]:
lcs_gen_impl5(2, 3) |> cleanup

quote
    r_1_1 = if s1[1] == s2[1]
            1
        else
            0
        end
    r_1_2 = if s1[1] == s2[2]
            1
        else
            r_1_1
        end
    r_2_1 = if s1[2] == s2[1]
            1
        else
            r_1_1
        end
    r_2_2 = if s1[2] == s2[2]
            1 + r_1_1
        else
            max(r_2_1, r_1_2)
        end
    r_1_3 = if s1[1] == s2[3]
            1
        else
            r_1_2
        end
    r_2_3 = if s1[2] == s2[3]
            1 + r_1_2
        else
            max(r_2_2, r_1_3)
        end
    return r_2_3
end

In [29]:
@generated function lcs_gen5(::Val{i}, ::Val{j}, s1, s2) where {i,j}
    lcs_gen_impl5(i, j)
end

lcs_gen5 (generic function with 1 method)

In [30]:
function lcs_gen5(s1, s2)
    lcs_gen5(Val(length(s1)), Val(length(s2)), s1, s2)
end

lcs_gen5 (generic function with 2 methods)

In [31]:
@test lcs_gen5("ABC", "ACD") == 2
# # The longest common subsequence is “GTAB”.
@test lcs_gen5("AGGTAB", "GXTXAYB") == 4
@test lcs_gen5("ABC", "CBA") == 1

Test Passed